In [1]:
import getpass
# And pyspark.sql to get the spark session
from pathlib import Path
import pickle
import os
import pandas as pd
import pickle
import os
import time
import numpy as np
from tqdm import tqdm_notebook as tqdm 
import matplotlib.pyplot as plt 
import annoy

# Python3 program to demonstrate
# the use of sample() function .
  
# import random 
from random import sample

In [2]:
user_vec = pd.read_parquet("user_vec_large.parquet",engine = 'pyarrow')

In [3]:
item_vec =  pd.read_parquet("item_vec_large.parquet",engine = 'pyarrow')

In [4]:
pwd

'/home/rz2432/final-project-group_101/CheckPoint Submission/fast_search'

In [5]:
ratings_full_train = pd.read_csv('/home/rz2432/final-project-group_101/CheckPoint Submission/ratings_full_train.csv', names = ('userId', 'movieId', 'rating', 'timestamp'))
ratings_full_val = pd.read_csv('/home/rz2432/final-project-group_101/CheckPoint Submission/ratings_full_val.csv', names = ('userId', 'movieId', 'rating', 'timestamp', "median_timestamp"))
ratings_full_test = pd.read_csv('/home/rz2432/final-project-group_101/CheckPoint Submission/ratings_full_test.csv', names = ('userId', 'movieId', 'rating', 'timestamp', "median_timestamp"))

ratings_full_val = ratings_full_val.drop(columns=['median_timestamp'])
ratings_full_test = ratings_full_test.drop(columns=['median_timestamp'])
ratings_full_train = ratings_full_train.drop(columns=['timestamp'])
ratings_full_val = ratings_full_val.drop(columns=['timestamp'])
ratings_full_test = ratings_full_test.drop(columns=['timestamp'])

In [6]:
total_item_user = pd.concat([ratings_full_train, ratings_full_test, ratings_full_val]).drop_duplicates()
#sort by ratings
total_item_user = total_item_user.sort_values(['rating'])
total_item_user['new_movieId'] = (total_item_user.groupby(['movieId'], sort=False).ngroup()+1)

In [7]:
label = total_item_user.groupby('userId')['movieId'].apply(list).to_numpy()

In [8]:
user = user_vec['features'].to_numpy()
item = item_vec['features'].to_numpy()

In [9]:
item = np.array([x for x in item]) 
user = np.array([x for x in user]) 

In [10]:
true_label = np.array([x for x in label]) 

<ipython-input-10-2127fb04794e>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  true_label = np.array([x for x in label])


In [ ]:
#sampling

In [11]:
def query_one_user(item,user,query,k):
    start = time.time()
    res = np.argsort(-item.dot(query))[:k]
    time_taken = time.time() - start
    return res,time_taken

In [12]:
query_one_user(item,user,user[600],10)

(array([17174, 12833, 18192, 38356,  9317, 26396,  9313, 21390, 34097,
        38351]),
 0.004630088806152344)

In [20]:
def precision(pred, label):
    count = 0
    for i in range(len(pred)):
        if pred[i] in label:
            count +=1
    return count/len(pred)

In [27]:
def brutal_search(items,users,k,label,user_sample):
    n_items = item.shape[0]
    n_feat = item.shape[1]
    n_users = user_sample.shape[0]
    
    time_list = []
    result_list = []
    acc = []
    
    for i in tqdm(range(n_users)):
        neighbours,time_taken = query_one_user(items,users,users[i],k)
        prec = precision(neighbours, label[i])
        time_list.append(time_taken) 
        result_list.append(neighbours)
        acc.append(prec)
        

    avg_time = sum(time_list) / len(time_list) 
    accu = sum(acc) / len(acc) 
    print(avg_time)
    print('brute search time:%.6f' % avg_time)
    print('accuracy:%.6f' % accu)
    
    return avg_time,result_list

In [32]:
#sample the first 10000 users, otherwise system collapse for brute search 
time_b,result_b = brutal_search(item,user,100,true_label[:10000],user[:10000])

<ipython-input-27-79ae09858715>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(n_users)):


  0%|          | 0/10000 [00:00<?, ?it/s]

0.004413980746269226
brute search time:0.004414
accuracy:0.002620


### Fast search with annoy

In [35]:
def build_tree(item,n_trees):
    f = len(item[0])
    t = annoy.AnnoyIndex(f, metric='dot') 
    for i in range(len(item)):
        t.add_item(i, item[i])
    t.build(n_trees)
    return t

In [36]:
def wrapup(obj, method, mapping): 
    get_map = lambda x: [x[mapping[i]] for i in range(len(mapping))]
    def wrapped(*args, **kwrds):
        return obj.__getattribute__(method)(*get_map(args)) 
    return wrapped

In [37]:
def fast_query_one_user(item,user,query,k,param,model_wrapped):
    start = time.time()
    res = model_wrapped(query, k, 100) 
    interval = time.time() - start
    return interval, res

In [38]:
tree = build_tree(item,100)
wrapped = wrapup(tree, 'get_nns_by_vector', [0, 1, 2])
time_taken,result = fast_query_one_user(item,user,user[0],10,[0,1,2],wrapped)
print(time_taken,result)

0.00015616416931152344 [2019, 13219, 8635, 38750, 8593, 29813, 35337, 38316, 10793, 26280]


In [39]:
def precision(pred, label):
    count = 0
    for i in range(len(pred)):
        if pred[i] in label:
            count +=1
    return count/len(pred)

In [40]:
precision(result, true_label[0])

0.0

In [41]:
def fast_search_with_annoy(item,user,label,model_wrapped,k,sample_user):
    n_items = item.shape[0]
    n_feat = item.shape[1]
    n_users = sample_user.shape[0]
    
    times = []
    precisions = []
    results = []

    for i in tqdm(range(n_users)):
        interval, res = fast_query_one_user(item,user,user[i],k,[0,1,2],model_wrapped)
        prec = precision(res, label[i]) 
        results.append(res)
        times.append(interval) 
        precisions.append(prec)

    mean_time = sum(times) / len(times) 
    mean_precision = sum(precisions) / len(precisions)
    print('-' * 26)
    print('Mean Query Search Time: %.6f' % mean_time) 
    print('Mean precision: %.6f' % mean_precision)
    return mean_time, mean_precision,results

In [43]:
tree = build_tree(item,100)
wrapped = wrapup(tree, 'get_nns_by_vector', [0, 1, 2])
proces_time, accuracy, result = fast_search_with_annoy(item,user,true_label, wrapped, 100,user) 

<ipython-input-42-8c63e029819a>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(n_users)):


  0%|          | 0/274198 [00:00<?, ?it/s]

--------------------------
Mean Query Search Time: 0.000070
Mean precision: 0.001866


In [45]:
n_trees = [10,100,200,500,700,1000]

loop_time = []
loop_precision = []
loop_res = []
selection = []
for k in n_trees:
    tree = build_tree(item,k)
    wrapped = wrapup(tree, 'get_nns_by_vector', [0, 1, 2])
    proces_time, accuracy, result = fast_search_with_annoy(item,user,true_label, wrapped, 100,user)
    selection.append(k)
    loop_time.append(proces_time)
    loop_precision.append(accuracy)
    loop_res.append(result)

<ipython-input-42-8c63e029819a>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(n_users)):


  0%|          | 0/274198 [00:00<?, ?it/s]

--------------------------
Mean Query Search Time: 0.000049
Mean precision: 0.001678


  0%|          | 0/274198 [00:00<?, ?it/s]

--------------------------
Mean Query Search Time: 0.000068
Mean precision: 0.001866


...

--------------------------
Mean Query Search Time: 0.000204
Mean precision: 0.005164


  0%|          | 0/274198 [00:00<?, ?it/s]

--------------------------
Mean Query Search Time: 0.000297
Mean precision: 0.005164


  0%|          | 0/274198 [00:00<?, ?it/s]

--------------------------
Mean Query Search Time: 0.000467
Mean precision: 0.005164


In [46]:
evaluation = {"params":selection,"times":loop_time,"accuracy":loop_precision}

In [47]:
pd.DataFrame(evaluation)

,params,times,accuracy
0,10,0.000049,0.001678
1,100,0.000068,0.001866
2,200,0.000106,0.005166
3,500,0.000204,0.005164
4,700,0.000297,0.005164
5,1000,0.000467,0.005164


In [49]:
#tree = 200
tree = build_tree(item,200)
wrapped = wrapup(tree, 'get_nns_by_vector', [0, 1, 2])
proces_time, accuracy, result = fast_search_with_annoy(item,user,true_label, wrapped, 100,user)

<ipython-input-42-8c63e029819a>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(n_users)):


  0%|          | 0/274198 [00:00<?, ?it/s]

--------------------------
Mean Query Search Time: 0.000108
Mean precision: 0.005166


In [42]:
#sample using 10000 user
tree = build_tree(item,200)
wrapped = wrapup(tree, 'get_nns_by_vector', [0, 1, 2])
proces_time, accuracy, result = fast_search_with_annoy(item,user,true_label[:10000], wrapped, 100,user[:10000])

<ipython-input-41-e31ed13b8dbd>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm(range(n_users)):


  0%|          | 0/10000 [00:00<?, ?it/s]

--------------------------
Mean Query Search Time: 0.000089
Mean precision: 0.005133
